In [2]:
import requests, time, csv, os
import random
from datetime import datetime, timedelta
import cloudinary
import cloudinary.uploader
from googletrans import Translator
import pandas as pd
from ctgan import CTGAN
import ast

In [3]:
translator = Translator()

translate_cache = {}

def translate_vi_to_en(text):
    if not text:
        return text
    if text in translate_cache:
        return translate_cache[text]
    try:
        result = translator.translate(text, src="vi", dest="en")
        translate_cache[text] = result.text
        time.sleep(0.15)
        return result.text
    except:
        return text

cloudinary.config(
  cloud_name = "dvxmaiofh", 
  api_key = "834668983718514", 
  api_secret = "KI_mVAdKhFNvtbJN4w9TWSzJqno",
  secure = True
)

CATEGORY_ID = 53530
MAX_PRODUCTS = 100

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

items = []
page = 1
MAX_PAGE = 150

def random_date(start_year=2022, end_year=2025):
    start = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31)
    return (start + timedelta(
        days=random.randint(0, (end - start).days)
    )).strftime("%Y-%m-%d")

FAKE_SPECIES = [
  "dog",
  "cate",
  "bird",
  "rabbit"
]

def sql_escape(value):
    if value is None:
        return "NULL"
    return "'" + str(value).replace("'", "''") + "'"

def write_sql(items):
    filename = f"insert_food.sql"
    with open(filename, "w", encoding="utf-8") as f:
        for item in items:
            f.write("-- ================================\n")
            f.write(f"-- {item['product_name']}\n")

            # 1. Product
            f.write("WITH new_product AS (\n")
            f.write("  INSERT INTO product (\n")
            f.write("    product_name, price, manufacture_date, entry_date, expiry_date, stock, type\n")
            f.write("  ) VALUES (\n")
            f.write(f"    {sql_escape(item['product_name'])},\n")
            f.write(f"    {item['price']},\n")
            f.write(f"    {sql_escape(item['manufacture_date'])},\n")
            f.write(f"    {sql_escape(item['entry_date'])},\n")
            f.write(f"    {sql_escape(item['expiry_date'])},\n")
            f.write(f"    {item['stock']},\n")
            f.write(f"    {sql_escape(item['type'])}\n")
            f.write("  ) RETURNING product_id\n")
            f.write(")")

            # 2. Product images (nếu có)
            imgList = item.get("imgList", [])
            if imgList:
                for idx, img in enumerate(imgList):
                    sep = "," if idx == 0 else ","
                    f.write(f"{sep}\n")
                    f.write(
                        f"img_{idx} AS (\n"
                        "  INSERT INTO productimage (product_id, image_url)\n"
                        f"  SELECT product_id, {sql_escape(img)} FROM new_product\n"
                        ")\n"
                    )

            # 3. Food
            f.write(
                "INSERT INTO food (product_id, weight, species, nutrition_description)\n"
                "SELECT product_id, "
                f"{sql_escape(item['weight'])}, "
                f"{sql_escape(item['species'])}, "
                f"{sql_escape(item['nutrition_description'])} "
                "FROM new_product;\n\n"
            )

    print(f"💾 Đã lưu {len(items)} item vào {filename}")

In [4]:
while len(items) < MAX_PRODUCTS:
    LIST_API = f"https://tiki.vn/api/v2/products?category={CATEGORY_ID}&page={page}"
    resp = requests.get(LIST_API, headers=headers).json()
    data = resp.get("data", [])

    if not data:
        print(f"[WARN] Page {page} has no data, skipping...")
        page += 1
        continue

    for p in data:
        if len(items) >= MAX_PRODUCTS:
            break

        pid = p["id"]
        detail = requests.get(f"https://tiki.vn/api/v2/products/{pid}", headers=headers).json()

        product_name = detail.get("name")
        price = detail.get("price")
        stock = random.randint(50, 200)
        type_ = "accessory"

        imgList = [img["base_url"] for img in detail.get("images", []) if "base_url" in img][:3]
        if not imgList:
            imgList = ["https://dummyimage.com/600x600"]

        manufacture_date = random_date(2022, 2024)
        entry_date = random_date(2024, 2025)
        expiry_date = random_date(2026, 2028)

        species = random.choice(FAKE_SPECIES)
        weight = random.randint(1, 20) * 0.5  # in kg
        nutrition_description = detail.get("short_description") or f"High quality {species} food with balanced nutrition."

        items.append({
            "product_name": translate_vi_to_en(product_name),
            "price": price,
            "manufacture_date": manufacture_date,
            "entry_date": entry_date,
            "expiry_date": expiry_date,
            "stock": stock,
            "type": type_,
            "imgList": imgList,
            "species": species,
            "weight": weight,
            "nutrition_description": nutrition_description
        })

        print(f"GET {len(items)} | {product_name[:40]}")

    page += 1

GET 1 | THỨC ĂN HẠT DÀNH CHO MÈO ZOI CAT TÚI 1Kg
GET 2 | hạt kê bóng đỏ nhiều chất dinh dưỡng làm
GET 3 | Hạt kê vàng (bóng vàng) dùng làm thức ăn
GET 4 | Hạt kê đỏ to (bóng đỏ) dùng làm thức ăn 
GET 5 | hạt đại mạch dùng làm thức ăn đầy đủ din
GET 6 | hạt trộn kèm hạt trái cây dành cho vẹt S
GET 7 | Hạt kê trộn nhiều chất dinh dưỡng cho ch
GET 8 | 100g - GÀ SẤY - THỊT SẤY - THỊT VIÊN ĂN 
GET 9 | Thức Ăn Cho Mèo Whiskas Adult 1+ YearsVị
GET 10 | Thức Ăn Cho Mèo Con Whiskas Junior 2-12 
GET 11 | Thức Ăn Cho Mèo Con Whiskas Junior 2-12 
GET 12 | Thức Ăn Cho Mèo Whiskas Adult 1+ Years V
GET 13 | [Combo 12 Gói] Thức Ăn Cho Mèo WHISKAS 1
GET 14 | [Combo 12 Gói] Thức Ăn Cho Mèo WHISKAS 1
GET 15 | [Combo 12 Gói] Thức Ăn Cho Mèo Con WHISK
GET 16 | [Combo 12 Gói] Thức Ăn Cho Mèo WHISKAS 1
GET 17 | [Combo 12 Gói] Thức Ăn Cho Mèo WHISKAS 1
GET 18 | Thức Ăn Cho Mèo Whiskas Adult 1+ Years V
GET 19 | [Combo 12 Gói] Thức Ăn Cho Mèo WHISKAS 1
GET 20 | Thức ăn hạt cho chó hỗ trợ xương khớp - 
GET 21 | 

In [5]:
print(items)
print(len(items))
for idx, row in enumerate(items):
    img_count = len(row.get("imgList", [])) if isinstance(row.get("imgList", []), list) else 0
    print(f"Sản phẩm {idx+1}: {img_count} ảnh")

[{'product_name': 'ZOI CAT SEED FOOD FOR CATS 1KG BAG', 'price': 58000, 'manufacture_date': '2024-08-05', 'entry_date': '2024-10-08', 'expiry_date': '2026-01-17', 'stock': 108, 'type': 'accessory', 'imgList': ['https://salt.tikicdn.com/ts/product/e3/2f/31/28c2240411d280255eac5e0ed2b27f36.jpg', 'https://salt.tikicdn.com/ts/product/70/e6/94/ea6bdb972d709438d0cc6489677639d4.jpg', 'https://salt.tikicdn.com/ts/product/13/f5/88/354bfec915eb7c3d9b80d7cd161523e4.jpg'], 'species': 'rabbit', 'weight': 2.5, 'nutrition_description': 'ZOICAT - Thức Ăn Hạt Khô Dành Cho Mèo Trưởng Thành Xuất sứ: Thái Lan. ĐẶC ĐIỂM: - Công thức natri thấp giảm nguy cơ mắc bệnh huyết áp, tim mạch, thận. - Bảo vệ răng và nướu nhờ giảm nguy cơ gây nên tì...'}, {'product_name': 'Red millet seeds are rich in nutrients and can be used as food for birds and parrots', 'price': 17999, 'manufacture_date': '2023-01-06', 'entry_date': '2024-01-17', 'expiry_date': '2028-08-08', 'stock': 146, 'type': 'accessory', 'imgList': ['https

In [6]:
# 1. Load dataset gốc (100 sản phẩm)
df = pd.DataFrame(items)  # hoặc pd.read_csv("accessory_products.csv")

# Nếu imgList là list, SDV không handle trực tiếp -> convert sang string
df['imgList'] = df['imgList'].apply(lambda x: ";".join(x) if isinstance(x, list) else x)

# 2. Khởi tạo model CTGAN
model = CTGAN(epochs=500)  # tăng epochs nếu dataset nhỏ để model học tốt hơn

# Convert dates sang string
for col in ["manufacture_date", "entry_date", "expiry_date"]:
    df[col] = df[col].astype(str)

# Convert imgList sang string
df['imgList'] = df['imgList'].apply(lambda x: ";".join(x) if isinstance(x, list) else x)

# Các cột categorical
discrete_columns = ["product_name", "type", "weight", "species", "nutrition_description",
                    "manufacture_date", "entry_date", "expiry_date", "imgList"]

# Khởi tạo model
model = CTGAN(epochs=500)

# Fit
model.fit(df, discrete_columns=discrete_columns)

# 4. Generate thêm synthetic data
synthetic_df = model.sample(3900)  # tạo thêm 3900 để tổng ~4000

def fix_imglist(imglist):
    if isinstance(imglist, str):
        imgs = [i for i in imglist.split(";") if i.strip()]
    elif isinstance(imglist, list):
        imgs = [i for i in imglist if i]
    else:
        imgs = []

    if not imgs:
        imgs = ["https://dummyimage.com/600x600"]
    return imgs

# Áp dụng fix_imglist cho cả df gốc
df['imgList'] = df['imgList'].apply(fix_imglist)

# Áp dụng fix_imglist cho synthetic
synthetic_df['imgList'] = synthetic_df['imgList'].apply(fix_imglist)

# 6. Gộp dữ liệu gốc + synthetic
full_df = pd.concat([df, synthetic_df], ignore_index=True)

for col in ["manufacture_date", "entry_date", "expiry_date"]:
    full_df[col] = pd.to_datetime(full_df[col], errors='coerce')

In [7]:
print(len(full_df))  # Kiểm tra tổng số sản phẩm
for idx, row in full_df.iterrows():
    img_count = len(row["imgList"]) if isinstance(row["imgList"], list) else 0
    print(f"Sản phẩm {idx+1}: {img_count} ảnh")

4000
Sản phẩm 1: 3 ảnh
Sản phẩm 2: 1 ảnh
Sản phẩm 3: 1 ảnh
Sản phẩm 4: 1 ảnh
Sản phẩm 5: 1 ảnh
Sản phẩm 6: 1 ảnh
Sản phẩm 7: 1 ảnh
Sản phẩm 8: 1 ảnh
Sản phẩm 9: 3 ảnh
Sản phẩm 10: 3 ảnh
Sản phẩm 11: 3 ảnh
Sản phẩm 12: 3 ảnh
Sản phẩm 13: 3 ảnh
Sản phẩm 14: 3 ảnh
Sản phẩm 15: 3 ảnh
Sản phẩm 16: 3 ảnh
Sản phẩm 17: 3 ảnh
Sản phẩm 18: 3 ảnh
Sản phẩm 19: 3 ảnh
Sản phẩm 20: 1 ảnh
Sản phẩm 21: 2 ảnh
Sản phẩm 22: 1 ảnh
Sản phẩm 23: 1 ảnh
Sản phẩm 24: 1 ảnh
Sản phẩm 25: 3 ảnh
Sản phẩm 26: 3 ảnh
Sản phẩm 27: 3 ảnh
Sản phẩm 28: 3 ảnh
Sản phẩm 29: 3 ảnh
Sản phẩm 30: 3 ảnh
Sản phẩm 31: 1 ảnh
Sản phẩm 32: 3 ảnh
Sản phẩm 33: 3 ảnh
Sản phẩm 34: 3 ảnh
Sản phẩm 35: 2 ảnh
Sản phẩm 36: 2 ảnh
Sản phẩm 37: 3 ảnh
Sản phẩm 38: 3 ảnh
Sản phẩm 39: 3 ảnh
Sản phẩm 40: 3 ảnh
Sản phẩm 41: 2 ảnh
Sản phẩm 42: 3 ảnh
Sản phẩm 43: 1 ảnh
Sản phẩm 44: 3 ảnh
Sản phẩm 45: 3 ảnh
Sản phẩm 46: 3 ảnh
Sản phẩm 47: 1 ảnh
Sản phẩm 48: 1 ảnh
Sản phẩm 49: 1 ảnh
Sản phẩm 50: 3 ảnh
Sản phẩm 51: 3 ảnh
Sản phẩm 52: 3 ảnh
Sản phẩm 53: 3 ả

In [8]:
# ==== Sau khi loop xong toàn bộ, ghi SQL hoặc CSV 1 lần ====
write_sql(full_df.to_dict(orient="records"))
full_df.to_csv("food_products_4000.csv", index=False, encoding='utf-8')
print(f"[INFO] Đã thu thập tổng cộng {len(full_df)} sản phẩm")

💾 Đã lưu 4000 item vào insert_food.sql
[INFO] Đã thu thập tổng cộng 4000 sản phẩm


In [9]:
import psycopg2

conn = psycopg2.connect(
    host="aws-1-ap-south-1.pooler.supabase.com",
    database="postgres",
    user="postgres.rruavcjmtgpxyznzwkhw",
    password="khaibaolocnguyen",
    port=5432
)
cur = conn.cursor()

with open("insert_food.sql", "r", encoding="utf-8") as f:
    sql = f.read()

cur.execute(sql)
conn.commit()
cur.close()
conn.close()
